In [1]:
import os
def load_polybench(folder, subfolders):
    polybench = {}
    for each in subfolders:
        for subdir, dirs, files in os.walk(folder + os.sep + each):
            for file in files:
                if (file.endswith(".c") or file.endswith(".h")):
                    if (file[:-2] not in polybench):
                        polybench[file[:-2]] = {}
                    polybench[file[:-2]][file[-1]] = subdir + os.sep + file
    return polybench
def show_name(filename):
    return filename.split(os.sep)[-1]

In [2]:
folder = 'polybench-c-4.2.1-beta'
subfolders = ['datamining','linear-algebra','medley','stencils']
polybench = load_polybench(folder, subfolders)

In [3]:
for k,v in polybench.items():
    print("Program: {:<15} [{}, {}]".format(k, show_name(v["c"]), show_name(v["h"])))

Program: correlation     [correlation.c, correlation.h]
Program: covariance      [covariance.c, covariance.h]
Program: gemm            [gemm.c, gemm.h]
Program: gemver          [gemver.c, gemver.h]
Program: gesummv         [gesummv.c, gesummv.h]
Program: symm            [symm.c, symm.h]
Program: syr2k           [syr2k.c, syr2k.h]
Program: syrk            [syrk.c, syrk.h]
Program: trmm            [trmm.c, trmm.h]
Program: 2mm             m.c, 2mm.h]
Program: 3mm             m.c, 3mm.h]
Program: atax            [atax.c, atax.h]
Program: bicg            [bicg.c, bicg.h]
Program: doitgen         [doitgen.c, doitgen.h]
Program: mvt             [mvt.c, mvt.h]
Program: cholesky        [cholesky.c, cholesky.h]
Program: durbin          [durbin.c, durbin.h]
Program: gramschmidt     [gramschmidt.c, gramschmidt.h]
Program: lu              [lu.c, lu.h]
Program: ludcmp          [ludcmp.c, ludcmp.h]
Program: trisolv         [trisolv.c, trisolv.h]
Program: deriche         [deriche.c, deriche.h]
Progra

In [4]:
def clean(filename):
    with open(filename, encoding='utf-8') as txt_file:
        raw_text = txt_file.read()
        while('*/' in raw_text):
            comment_start = raw_text.index('/*')
            comment_end = raw_text.index('*/')
            raw_text = raw_text[:comment_start] + raw_text[comment_end+2:]
        main_start = raw_text.index('int main')
        raw_text = raw_text[:main_start]
        rows = [x.strip() for x in raw_text.split('\n')]
        rows = list(filter(lambda x: len(x) > 0, rows))
        rows = list(filter(lambda x: "#" not in x, rows))
        rows = [(row[:row.index('//')] if ('//' in row) else row) for row in rows]
        rows = list(filter(lambda x: "POLYBENCH_DUMP_START;" not in x, rows))
        rows = list(filter(lambda x: "POLYBENCH_DUMP_BEGIN" not in x, rows))
        rows = list(filter(lambda x: "POLYBENCH_DUMP_END" not in x, rows))
        rows = list(filter(lambda x: "POLYBENCH_DUMP_FINISH;" not in x, rows))
        rows = list(filter(lambda x: "POLYBENCH_2D_ARRAY_DECL" not in x, rows))
        text = ' '.join(rows)
    return text
def replace(text, filename):
    text = text.replace('  ', ' ')
    text = text.replace('DATA_TYPE', 'float')
    text = text.replace('DATA_PRINTF_MODIFIER', '"%0.2f "')
    text = text.replace('SCALAR_VAL', '')
    text = text.replace('SQRT_FUN', 'sqrt')
    text = text.replace('EXP_FUN', 'exp')
    text = text.replace('POW_FUN', 'pow')
    text = text.replace('POLYBENCH_ARRAY', '*')
    text = text.replace('POLYBENCH_FREE_ARRAY', '(void*)')
    text = text.replace('POLYBENCH_DUMP_TARGET', 'stderr')
    return text

In [5]:
from c_dict import string_to_dict

def generate_ast(polybench):
    for k,v in polybench.items():
        v['text'] = None
        v['ast'] = None
        try:
            print('Program: {:<15} Source code...clean...'.format(k), end='')
            text = clean(v['c'])
            print('replace...', end='')
            text = replace(text, v['h'])
            print('Generate AST...', end='')
            ast_dict = string_to_dict(text)
            print('depurate...', end='')
            ast_dict = depurate_ast(ast_dict, False)
            print('OK!')
        except:
            print('NO!')
        else:
            v['text'] = text
            v['ast'] = ast_dict

In [6]:
wrangling ={'FileAST':{'pop':['coord'],'list':'ext'},
            'FuncDef':{'pop':['coord','param_decls'],'dict':['decl','body']},
            'Decl':{'pop':['quals','align','funcspec','coord','bitsize'],'dict':['type']},
            'FuncDecl':{'pop':['coord','args'],'dict':['type']},
            'TypeDecl':{'pop':['quals','align','coord'],'dict':['type']},
            'IdentifierType':{'pop':['coord']},
            'Compound':{'pop':['coord'],'list':'block_items'},
            'Assignment':{'pop':['coord'],'dict':['lvalue','rvalue']},
            'UnaryOp':{'pop':['coord'],'dict':['expr']},
            'ID':{'pop':['coord']},
            'Cast':{'pop':['coord','to_type'],'dict':['expr']},
            'For':{'pop':['coord','init','cond','next'],'dict':['stmt']},
            'ArrayRef':{'pop':['coord'],'dict':['name','subscript']},
            'BinaryOp':{'pop':['coord'],'dict':['left','right']},
            'If':{'pop':['coord'],'dict':['cond','iftrue','iffalse']},
            'Constant':{'pop':['coord']},
            'FuncCall':{'pop':['coord'],'dict':['name','args']},
            'ExprList':{'pop':['coord'],'list':'exprs'},
            'TernaryOp':{'pop':['coord'],'dict':['cond','iftrue','iffalse']},
            'ArrayDecl':{'pop':['coord','dim_quals'],'dict':['type','dim']},
            'Typedef':{'pop':['coord','quals','storage'],'dict':['type']}}
def depurate_ast(ast, verbose, whitespaces=''):
    print('\n{}Inside of {}'.format(whitespaces,ast['_nodetype']) if verbose else '',end='')
    try:
        type_s = ast['_nodetype']
        for x in wrangling[type_s]['pop']:
            ast.pop(x)
        if ('list' in wrangling[type_s]):
            list_s = wrangling[type_s]['list']
            for i in range(len(ast[list_s])):
                ast[list_s][i] = depurate_ast(ast[list_s][i],verbose,whitespaces+'    ')
        if ('dict' in wrangling[type_s]):
            dict_l = wrangling[type_s]['dict']
            for i in range(len(dict_l)):
                if(ast[dict_l[i]] != None):
                    ast[dict_l[i]] = depurate_ast(ast[dict_l[i]],verbose,whitespaces+'    ')
    except:
        print('Error with {}!'.format(ast['_nodetype']))
    return ast

In [7]:
generate_ast(polybench)

Program: correlation     Source code...clean...replace...Generate AST...depurate...OK!
Program: covariance      Source code...clean...replace...Generate AST...depurate...OK!
Program: gemm            Source code...clean...replace...Generate AST...depurate...OK!
Program: gemver          Source code...clean...replace...Generate AST...depurate...OK!
Program: gesummv         Source code...clean...replace...Generate AST...depurate...OK!
Program: symm            Source code...clean...replace...Generate AST...depurate...OK!
Program: syr2k           Source code...clean...replace...Generate AST...depurate...OK!
Program: syrk            Source code...clean...replace...Generate AST...depurate...OK!
Program: trmm            Source code...clean...replace...Generate AST...depurate...OK!
Program: 2mm             Source code...clean...replace...Generate AST...depurate...OK!
Program: 3mm             Source code...clean...replace...Generate AST...depurate...OK!
Program: atax            Source code...clea

In [8]:
def show_depurate_ast(name):
    print(polybench[name]['text'], '\n')
    show_depurate_ast_r(polybench[name]['ast'])
    
def show_depurate_ast_r(ast, whitespaces=''):
    print('{}Inside of {}'.format(whitespaces,ast['_nodetype']))
    type_s = ast['_nodetype']
    if ('list' in wrangling[type_s]):
        list_s = wrangling[type_s]['list']
        for i in range(len(ast[list_s])):
            show_depurate_ast_r(ast[list_s][i],whitespaces+'    ')
    if ('dict' in wrangling[type_s]):
        dict_l = wrangling[type_s]['dict']
        for i in range(len(dict_l)):
            if(ast[dict_l[i]] != None):
                show_depurate_ast_r(ast[dict_l[i]],whitespaces+'    ')

In [9]:
show_depurate_ast('correlation')

static void init_array (int m, int n, float *float_n, float POLYBENCH_2D(data,N,M,n,m)) { int i, j; *float_n = (float)N; for (i = 0; i < N; i++) for (j = 0; j < M; j++) data[i][j] = (float)(i*j)/M + i; } static void print_array(int m, float POLYBENCH_2D(corr,M,M,m,m)) { int i, j; for (i = 0; i < m; i++) for (j = 0; j < m; j++) { if ((i * m + j) % 20 == 0) fprintf (stderr, "\n"); fprintf (stderr, "%0.2f ", corr[i][j]); } } static void kernel_correlation(int m, int n, float float_n, float POLYBENCH_2D(data,N,M,n,m), float POLYBENCH_2D(corr,M,M,m,m), float POLYBENCH_1D(mean,M,m), float POLYBENCH_1D(stddev,M,m)) { int i, j, k; float eps = (0.1); for (j = 0; j < _PB_M; j++) { mean[j] = (0.0); for (i = 0; i < _PB_N; i++) mean[j] += data[i][j]; mean[j] /= float_n; } for (j = 0; j < _PB_M; j++) { stddev[j] = (0.0); for (i = 0; i < _PB_N; i++) stddev[j] += (data[i][j] - mean[j]) * (data[i][j] - mean[j]); stddev[j] /= float_n; stddev[j] = sqrt(stddev[j]); stddev[j] = stddev[j] <= eps ? (1.0) : s

In [10]:
#search = polybench['nussinov']['ast']['ext'][0]
#print(type(search))
#print(search)

In [11]:
#for k,v in search.items():
#    print(k, v)